In [13]:
import numpy as np
import pandas as pd

data = pd.DataFrame({"Sốt": ["Cao ", "Cao", "Cao ", "Thấp", " Cao ", "Thấp", "Thấp", " Cao ", "Thấp", " Cao "] ,
                    "Ho": ["Có", "Không", "Có", "Có", "Có", "Không", "Có", "Không", "Không", "Có"] ,
                    "Đau họng": ["Có", "Có", "Không", "Có", "Có", "Có", "Không", "Có", "Không", "Có"] ,
                    "Mệt mỏi": ["Có", "Không", "Có", "Có", "Không", "Không", "Có", "Có", "Không", "Có"] ,
                    "Flu": ["Có", "Có", "Có", "Không", "Có", "Không", "Không", "Có", "Không", "Có"]
                   })


print(data)


     Sốt     Ho Đau họng Mệt mỏi    Flu
0   Cao      Có       Có      Có     Có
1    Cao  Không       Có   Không     Có
2   Cao      Có    Không      Có     Có
3   Thấp     Có       Có      Có  Không
4   Cao      Có       Có   Không     Có
5   Thấp  Không       Có   Không  Không
6   Thấp     Có    Không      Có  Không
7   Cao   Không       Có      Có     Có
8   Thấp  Không    Không   Không  Không
9   Cao      Có       Có      Có     Có


In [9]:
print(data[data["Flu"] == "Có"])

     Sốt     Ho Đau họng Mệt mỏi Flu
0   Cao      Có       Có      Có  Có
1    Cao  Không       Có   Không  Có
2   Cao      Có    Không      Có  Có
4   Cao      Có       Có   Không  Có
7   Cao   Không       Có      Có  Có
9   Cao      Có       Có      Có  Có


In [14]:
# Dữ liệu cần dự đoán
new_patient = {"Sốt" : "Cao",
                "Ho" : "Có",
                "Đau họng" : "Có",
                "Mệt mỏi" : "Không"
                } 

# Tính xác suất tiên nghiệm P(C)
P_flu = data["Flu"].value_counts(normalize=True)["Có"] #P(C=1)
P_not_flu = 1 - P_flu #P(C=0)
print(f"P_flu: {P_flu}")
print(f"P_not_flu: {P_not_flu}")

P_flu: 0.6
P_not_flu: 0.4


In [16]:
# Hàm tính xác suất có điều kiện P(x_i|C) với Laplace smoothing
def condition_prob(feature, value, flu_status):
    subset = data[data["Flu"] == flu_status]
    count_features = len(subset[subset[feature] == value]) + 1
    total_count = len(subset) + 2
    return count_features / total_count

# P(X|C=1)
P_X_given_flu = 1
# P(X|C=0)
P_X_given_not_flu = 1

for feature, value in new_patient.items():
    prob_given_flu = condition_prob(feature, value, "Có")
    prob_given_not_flu = condition_prob(feature, value, "Không")

    P_X_given_flu *= prob_given_flu
    P_X_given_not_flu *= prob_given_not_flu
    
    print(f"P({feature}={value}|Flu=Có) = {prob_given_flu:.4f}")
    print(f"P({feature}={value}|Flu=Không) = {prob_given_not_flu:.4f}")

P(Sốt=Cao|Flu=Có) = 0.2500
P(Sốt=Cao|Flu=Không) = 0.1667
P(Ho=Có|Flu=Có) = 0.6250
P(Ho=Có|Flu=Không) = 0.5000
P(Đau họng=Có|Flu=Có) = 0.7500
P(Đau họng=Có|Flu=Không) = 0.5000
P(Mệt mỏi=Không|Flu=Có) = 0.3750
P(Mệt mỏi=Không|Flu=Không) = 0.5000


In [19]:
numerator_flu = P_X_given_flu * P_flu
numerator_not_flu = P_X_given_not_flu * P_not_flu

# Chuẩn hóa xác suất
total = numerator_flu + numerator_not_flu
P_flu_given_X = numerator_flu / total
P_not_flu_given_X = numerator_not_flu / total

print("\nKết quả Naive Bayes:")
print(f"P(Flu=Có|X) = {P_flu_given_X:.4f}")
print(f"P(Flu=Không|X) = {P_not_flu_given_X:.4f}")

print("\nKết luận:", "Bệnh nhân có khả năng bị cúm." if P_flu_given_X > P_not_flu_given_X else "Bệnh nhân KHÔNG có khả năng bị cúm.")


Kết quả Naive Bayes:
P(Flu=Có|X) = 0.8260
P(Flu=Không|X) = 0.1740

Kết luận: Bệnh nhân có khả năng bị cúm.
